[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Kent Law, 2021 ([t.law@se21.qmul.ac.uk](mailto:t.law@se21.qmul.ac.uk)). All rights reserved.

# DIY Covid-19 Dashboard

This is a my DIY Covid Dashboard, which used a software development kit provided by Public Health England and some other popular python libraries, such is pandas, matplotlib and ipywidgets. The dashboard is being displayed by using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. In this panel, it will show the Covid-19 cases, their distribution among different age bands and genders and the vaccination, as an overview of the coronvirus in England.

In [96]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [97]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [98]:
# At first, I classify the data I collected from the API into two subclass: trends and distribution
trends = ['case_trend_data', 'vaccination_trend_data']
distribution = ['age_distribution_data']

# Then, creating a general function to retrieve data from a 'canned' JSON file
def get_canned_data(data_set):
    with open(f"{data_set}.json") as INFILE:
        data = json.load(INFILE)
        dates = [grp['date'] for grp in data['data']]
        dates.sort()
        return [data, dates]

# Using the function, I can create three sets of data: case_trend_data, vaccination_trend_data, age_distribution_data
default_case_trend_data = get_canned_data(trends[0])
default_vaccination_trend_data = get_canned_data(trends[1])
default_age_distribution_data = get_canned_data(distribution[0])

# This is a helper function to allow dates to be sorted easily
def parse_date(dates_tring):
    return pd.to_datetime(dates_tring, format = "%Y-%m-%d")

# Now, I create a general function to create a trend data's dataframes
def gen_trend_df(trend_data):
    data = trend_data
    start_date = parse_date(data[1][0])
    end_date = parse_date(data[1][-1])
    index = pd.date_range(start_date, end_date, freq='D')
    columns = [column for column in data[0]['data'][0].keys()]
    columns.pop(0)
    df = pd.DataFrame(index = index, columns = columns)
    for grp in data[0]['data']:
        date = parse_date(grp['date'])
        for column in columns:
            if pd.isna(df.loc[date, column]):
                value= float(grp[column]) if grp[column]!= None else 0.0
                df.loc[date, column] = value
    df.fillna(0.0, inplace=True)
    return df

# Distribution data has a separate function which wraps all data handling and dataframe creation, due to its uniqueness
def gen_age_distribution_df(distribution_data):
    latest_distribution_data = distribution_data['data'][0]
    males = latest_distribution_data['males']
    females = latest_distribution_data['females']
    age_ranges = [entry['age'] for entry in males]
    def min_age(age_range):
        age_range = age_range.replace('+','')
        start = age_range.split('_')[0]
        return int(start)
    age_ranges.sort(key=min_age)
    df = pd.DataFrame(index = age_ranges, columns = ['males', 'females', 'total'])
    for entry in males:
        age_band = entry['age']
        df.loc[age_band, 'males'] = entry['value']
    for entry in females:
        age_band = entry['age']
        df.loc[age_band, 'females'] = entry['value']
    df['total'] = df['males'] + df['females']
    return df

# Dataframes are created, and they should be global variable as they would be refreshed in the future
case_trend_df = gen_trend_df(default_case_trend_data)
vaccination_trend_df = gen_trend_df(default_vaccination_trend_data)
age_distribution_df = gen_age_distribution_df(default_age_distribution_data[0])

This is a work done at 1st of Dec, 2021. By clicking the below button, it accesses the latest statistics immediately and refresh the graphs.

In [99]:
# I'm a bit lazy, therefore the refresh button will update all the graphs
# Therefore, I created will use the below function to update dataframes all together
def access_COV19API():
    # Define the structures of the data sets first
    filters = [
        'areaType=nation',
        'areaName=England'
        ]
    case_trend = {
        "date": "date",
        "daily_cases": "newCasesByPublishDate",
        "daily_admissions": "newAdmissions",
        "daily_deaths": "newDeaths28DaysByPublishDate"  
        }
    age_distribution = {
        "date": "date",
        "males": "maleCases",
        "females": "femaleCases"  
        }
    vaccination_trend = {
        "date": "date",
        "first_dose": "newPeopleVaccinatedFirstDoseByPublishDate",
        "second_dose": "newPeopleVaccinatedSecondDoseByPublishDate",
        "booster_dose": "newPeopleVaccinatedThirdInjectionByPublishDate"
        }
    structures = [case_trend, age_distribution, vaccination_trend]
    # Create APIs
    case_trend_api = Cov19API(filters=filters, structure=structures[0])
    age_distribution_api = Cov19API(filters=filters, structure=structures[1])
    vaccination_trend_api = Cov19API(filters=filters, structure=structures[2])
    # Create a function to manipulate the data into a suitable format for dataframe creation
    def pack_fresh_data(data):
        dates = [grp['date'] for grp in data['data']]
        dates.sort()
        return [data, dates]
    # Create data sets through calling the APIs and package them at the same time
    case_trend_data = pack_fresh_data(case_trend_api.get_json())
    age_distribution_data = pack_fresh_data(age_distribution_api.get_json())
    vaccination_trend_data = pack_fresh_data(vaccination_trend_api.get_json())
    # Output all the data sets
    return [case_trend_data, vaccination_trend_data, age_distribution_data]


In [100]:
# This is a button callback which executes when the user clicks the button
def api_button_callback(button):
    try:
        # Get fresh data from the API
        covid_data = access_COV19API()
        # Declare global variables for referencing dataframes outside the current scope
        global case_trend_df
        global vaccination_trend_df
        global age_distribution_df
        # Overwriting the dataframes with new statistics
        case_trend_df = gen_trend_df(covid_data[0])
        vaccination_trend_df = gen_trend_df(covid_data[1])
        age_distribution_df = gen_age_distribution_df(covid_data[2][0])
        # Re-draw graphs
        redraw_graph()
        # If the update is successful, button icon should reflect and be disabled as it is the latest already
        refresh_button.icon = 'check'
        refresh_button.description = 'UPDATED'
        refresh_button.disabled = True 
    except:
        pass
    

# This is the button 
refresh_button = wdg.Button(
    description = 'UPDATE',
    disabled = False,
    button_style = 'danger',
    tooltip = "Click to refresh data",
    icon = 'download'
)

def redraw_graph():
    # Declare global variable for referencing outputs outside the current scope
    global case_trend_output
    global vaccination_trend_output
    global age_distribution_output
    global case_cols
    global vaccination_cols
    global age_cols
    # I try to clear the output first
    # .clear_put() method accepts argument "wait" to delay the clearance until the widget to have new thing to display
    # It enables smoother transition
    case_trend_output.clear_output(wait = True)
    vaccination_trend_output.clear_output(wait = True)
    age_distribution_output.clear_output(wait = True)
    # Then re-create a new output (tried but failed)
    # case_trend_output = wdg.interactive_output(
    #     case_graph,
    #     {'graph_cols': case_cols, 'graph_scale': case_scale}
    # )
    # vaccination_trend_output = wdg.interactive_output(
    #     vaccination_graph,
    #     {'graph_cols': vaccination_cols, 'graph_scale': vaccination_scale}
    # )
    # age_distribution_output = wdg.interactive_output(
    #     age_distribution_graph,
    #     {'graph_cols': age_cols}
    # )
    # Hard re-draw by tricking the widget
    case_cols.value = ['daily_cases', 'daily_admissions']
    case_cols.value = ['daily_cases', 'daily_admissions', 'daily_deaths']
    vaccination_cols.value = ['first_dose', 'second_dose']
    vaccination_cols.value = ['first_dose', 'second_dose', 'booster_dose']
    age_cols.value = ['males', 'females']
    age_cols.value = ['males', 'females', 'total']


refresh_button.on_click(api_button_callback)
display(refresh_button)



Button(button_style='danger', description='REFRESH', icon='download', style=ButtonStyle(), tooltip='Click to r…

## Graphs and Analysis
(The right hand side is a interactive control panel, user can switch between statistics and logarithms.)

The first graph captured the **daily cases, daily admissions and daily deaths** of COVID-19 in England. It shows that the last peak was from 2020 December to 2021 February and the pandemic is gtting more seriously recently. Luckily, the admissions and the deaths numbers are started to be less co-related with the case numbers than the previous peak. It probably reveals the effectiveness of vaccination.

In [ ]:
# Creating graph's output options
case_cols = wdg.SelectMultiple(
    options = ['daily_cases', 'daily_admissions', 'daily_deaths'],
    value = ['daily_cases', 'daily_admissions', 'daily_deaths'],
    rows = 3,
    description = 'Stats:',
    disabled = False
)
# Creating graph's logarithm options
case_scale = wdg.RadioButtons(
    options = ['linear', 'log'],
    description = 'Scale:',
    disabled = False
)
# Creating graph
def case_graph(graph_cols, graph_scale):
    if graph_scale == 'linear':
        log_scale = False
    else:
        log_scale = True
    n_cols = len(graph_cols)
    if n_cols > 0:
        case_trend_df[list(graph_cols)].plot(logy = log_scale)
        plt.show()
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
# Creating graph's output in the form of ipywidget
case_trend_output = wdg.interactive_output(
    case_graph,
    {'graph_cols': case_cols, 'graph_scale': case_scale}
)
case_controls = wdg.VBox([case_cols, case_scale])
# Cleaner format
case_form = wdg.HBox([case_trend_output, case_controls])
display(case_form)

The second graph captured the **daily number of vaccination: first dose, second dose and booster** during COVID-19 period in England. It shows that the vast majority of population has been vaccinated and a significant number of people are taking a boost or the third jab.

In [ ]:
# Codes are in the same order as the above
vaccination_cols = wdg.SelectMultiple(
    options = ['first_dose', 'second_dose', 'booster_dose'],
    value = ['first_dose', 'second_dose', 'booster_dose'],
    rows = 3,
    description = 'Stats:',
    disabled = False
)
vaccination_scale = wdg.RadioButtons(
    options = ['linear', 'log'],
    description = 'Scale:',
    disabled = False
)
def vaccination_graph(graph_cols, graph_scale):
    if graph_scale == 'linear':
        log_scale = False
    else:
        log_scale = True
    n_cols = len(graph_cols)
    if n_cols > 0:
        vaccination_trend_df[list(graph_cols)].plot(logy = log_scale)
        plt.show()
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
vaccination_trend_output = wdg.interactive_output(
    vaccination_graph,
    {'graph_cols': vaccination_cols, 'graph_scale': vaccination_scale}
)
vaccination_controls = wdg.VBox([vaccination_cols, vaccination_scale])
vaccination_form = wdg.HBox([vaccination_trend_output, vaccination_controls])
display(vaccination_form)

The third graph captured the **age distribution of cases divided by age bands** during COVID-19 period in England. Overrall, females are more likely to be infected than males and the coronavirus is extensively spread in the groups of children, teenagers and adults, instead of targeting a specific group. The reason behind could be the highly contagious property of the virus and hence it is common among the groups which have more outdoor and physical activities.

In [ ]:
# Codes are in the same order as the above
age_cols = wdg.SelectMultiple(
    options = ['males', 'females', 'total'],
    value = ['males', 'females', 'total'],
    rows = 3,
    description = 'Gender',
    disabled = False
)
def age_distribution_graph(graph_cols):
    n_cols = len(graph_cols)
    if n_cols > 0:
        age_distribution_df.plot(kind = 'bar', y = list(graph_cols))
        plt.show()
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
age_distribution_output = wdg.interactive_output(
    age_distribution_graph,
    {'graph_cols': age_cols}
)
age_controls = wdg.VBox([age_cols])
age_form = wdg.HBox([age_distribution_output, age_controls])
display(age_form)

## Thank you!

Thanks for viewing my mini dashboard. Wish the pandemic will be past soon and life returns to normal. Cheers!

**Author and Copyright Notice**:
Designed and developed by <u>Kent Law</u> 

**Acknowledgement**: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*